In [1]:
import sys
import pandas as pd
sys.path.append('D:/research/jokesclustering/') # ваш путь до корня проекта
from components import FasttextVectorizer, KmeansSimpleClusterExtractor
from components_simple_norm import SimpleNormVectorizer
from vector_clustering.abstract.model import Model
from vector_clustering.data.manager import get_jokes_as_dataframe, load_pandas_csv, load, ARANEUM_FASTTEXT
from sklearn.cluster import KMeans

In [2]:
jokes = get_jokes_as_dataframe()
jokes.head()

,joke_text
0,"Мужчину трудно задеть за живое, но уж если зад..."
1,В нашем кемпинге строго запрещено людям разног...
2,А как хорошо у девушек начинается: любимый: ми...
3,"Только у нас фраза ""поправить здоровье"" означа..."
4,"Одна белка случайно попробовала пиво и поняла,..."


### 1. Извлечем кластеры первым методом

Каждой шутке припишем название кластера в соответствии с кластеризацией на основе сумму векторов

In [4]:
from sklearn.externals import joblib
model = joblib.load('knn_fasttext.pkl')
jokes['sum_clustering'] = model.labels_

### 2. Извлечем кластеры вторым методом

Каждая шутка в данном подходе представима как нормированная сумма векторов слов

In [14]:
vectorizer = SimpleNormVectorizer(ARANEUM_FASTTEXT)
kmeans_model = KMeans(n_clusters=50, random_state=42)
cluster_extractor = KmeansSimpleClusterExtractor(kmeans_model)
model = Model(vectorizer, cluster_extractor)

In [15]:
texts = jokes.joke_text.values

In [16]:
demo = model.fit(texts)

vectorizer is fitted
clustering model is fitted


In [23]:
from sklearn.externals import joblib
joblib.dump(model.get_model(), 'kmeans_fasttext_normalised.pkl')

['kmeans_fasttext_normalised.pkl']

In [22]:
demo[41]

array(['Неудачник - тот, у кого и жена, и любовница имеют одинаковый месячный цикл.',
       'Если одинокая молодая соседка говорит вам «если что надо, заходите, не стесняйтесь», в первый же вечер заходите и не стесняйтесь',
       'Возвращается неожиданно мужик домой, запирает шкаф и говорит жене: - А теперь поговорим наедине.',
       'Спрашиваю мужа: «Ты поел?» Тот меня передразнивает: «Ты поел?» Я: «Хватит меня передразнивать» Муж снова: «Хватит меня передразнивать!» Я: «Я так тебя люблю, что подарю тебе шубу на день рождение!»  Муж: «Да поел я…»',
       'Почему жена у своей подружки может заночевать, а муж у своей нет?',
       'После скандала жена уехала к маме, так сказать "на истерическую родину".',
       'Муж возвращается домой после работы, на столе записка: Дорогой, я ушла к подруге, на ужин рыба, удочка в коридоре',
       'Жена мужу: — Станешь меня обижать, пойду на курсы "Как стать стервой" Муж: — Преподавателем?',
       'Сын спросил : - Мам, а у негритянки пудра чёрно

In [24]:
jokes['norm_clustering'] = model.get_model().labels_

In [25]:
jokes.joke_text.shape

(136338,)

In [68]:
jokes['id'] = [x for x in range(jokes.joke_text.shape[0])]

### Сравним, насколько похожи кластеры между двумя методами
Как часто происходит такое, что шутки, фактически, оказываются в одном и том же кластере независимо от метода

In [77]:
sum_clustering_association = [0] * jokes.sum_clustering.unique().shape[0]
for cluster_id in jokes.sum_clustering.unique():
    sum_clustering_association[cluster_id] = set(jokes[jokes.sum_clustering == cluster_id].id.values)


norm_clustering_association = [0] * jokes.norm_clustering.unique().shape[0]
for cluster_id in jokes.norm_clustering.unique():
    norm_clustering_association[cluster_id] = set(jokes[jokes.norm_clustering == cluster_id].id.values)

In [78]:
def jaccard(s1, s2):
    return len(s1.intersection(s2)) / len(s1.union(s2))

In [88]:
import numpy as np
def find_max_sim_jaccard_cluster(cluster_id):
    target_cluster = norm_clustering_association[cluster_id]
    similarities = [jaccard(target_cluster, x) for x in sum_clustering_association]
    max_sim_idx = np.argmax(similarities)
    max_sim = similarities[max_sim_idx]
    print('most similar for {} is {}, sim is: {}'.format(cluster_id, max_sim_idx, max_sim))
    return (max_sim_idx, max_sim)

In [89]:
similarities = [find_max_sim_jaccard_cluster(x) for x in range(jokes.sum_clustering.unique().shape[0])]

most similar for 0 is 5, sim is: 0.2742336371168186
most similar for 1 is 43, sim is: 0.07392915980230642
most similar for 2 is 27, sim is: 0.15671062839410396
most similar for 3 is 47, sim is: 0.06172839506172839
most similar for 4 is 20, sim is: 0.17769718948322757
most similar for 5 is 12, sim is: 0.08902877697841727
most similar for 6 is 29, sim is: 0.20810313075506445
most similar for 7 is 25, sim is: 0.1810850439882698
most similar for 8 is 34, sim is: 0.0897172236503856
most similar for 9 is 49, sim is: 0.12563736487864574
most similar for 10 is 28, sim is: 0.055072028811524606
most similar for 11 is 1, sim is: 0.04207841887153331
most similar for 12 is 19, sim is: 0.039089055064581914
most similar for 13 is 2, sim is: 0.33483483483483484
most similar for 14 is 1, sim is: 0.10415464512406232
most similar for 15 is 11, sim is: 0.09532095901005413
most similar for 16 is 46, sim is: 0.06821236559139784
most similar for 17 is 43, sim is: 0.0991194968553459
most similar for 18 is 45,

### Анализ кластеров 13 и 2 для каждого метода

In [102]:
norm_jokes = jokes[jokes.norm_clustering == 13]
sum_jokes = jokes[jokes.sum_clustering == 2]
norm_jokes.shape, sum_jokes.shape

((982, 4), (1685, 4))

In [105]:
norm_jokes_ids = norm_jokes.id.values
sum_jokes_ids = sum_jokes.id.values

In [122]:
left_diff = set(norm_jokes_ids) - set(sum_jokes_ids)
left_frame = norm_jokes[norm_jokes.id.apply(lambda x: x in left_diff)]
print(left_frame.shape)
examples = left_frame.joke_text.values
for v in examples[:20]:
    print(v)
    print()

(313, 4)
Дэвид Бекхем подписал контракт с ярославским Шинником во время раздачи автографов..

Конечно, в играх такого уровня, может проиграть любая команда, но РОССИЯ проиграть может только ЧЕМПИОНАМ!!!

После разговора по душам с главным тренером Станиславом Черчесовым нападающий «Спартака» Квинси попросил политического убежища в Англии.

Под видом велосипедной прогулки тренеры «Торпедо» бесплатно доставили игроков на очередной матч в Ярославль.

Во время матча с участием «Торпедо» со стадиона ушли даже кресла.

В Амстердаме открылся чемпионат мира по нетрадиционному ориентированию.

Роман Абрамович решил прикупить футбольный клуб просто для себя любимого:Рома.

Петербуржцы переименовали стадион Зенит-Арена в более актуальное Баклан-Арена.

Женская лыжная сборная США попросила у WADA разрешительных справок на допинг, пожаловавшись на слабую эрекцию.

Манит, манит, манит Куршавель Каждый год по замкнутому кругу...

Чемпион мира по бирюлькам может заниматься сексом, не разбудив при этом

In [121]:
right_diff = set(sum_jokes_ids) - set(norm_jokes_ids)
right_frame = sum_jokes[sum_jokes.id.apply(lambda x: x in right_diff)]
print(right_frame.shape)
examples = right_frame.joke_text.values
for v in examples[:20]:
    print(v)
    print()

(1016, 4)
У российских спортсменов появилось новое развлечение: Приехать на соревнования в Литву, занять там первое место и заставить свободолюбивых литовцев слушать запрещенную музыку.

Вот ты мне как спортсмен скажи: стрелы — стреляют, мечи — мечут, а дротики?

Официальным напитком чемпионата России по футболу является тормозная жидкость.

Перед противостоянием с Рахимичем и Дуду Динияр Билялетдинов написал завещание.

Многого ожидали от открытия игр в Пенине, но чтобы олимпийский огонь зажег ЛеНин....

Воробьи с Восточной улицы самым опасным игроком чемпионата России считают Романа Адамова.

футболисты сборной россии убедительно доказали, что они никогда в жизни не употребляли мельдоний

Полузащитник ЦСКА Даниэл Карвалью на тренировке пытался принять мяч на грудь, но не получилось: помешал второй подбородок.

На тренировке питерского «Зенита» по свистку тренера позже всех присел Анатолий Тимощук и таким образом стал капитаном команды.

На сессии МОК олимпийскую эстафету признали отд